MODEL I/O

In [1]:
#!pip install python_dotenv

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key=os.getenv("OPENAI_API_KEY")
pinecone_api_key=os.getenv("PINECONE_API_KEY")


In [3]:
#!pip install langchain_community

In [4]:
!pip install langchain_openai

In [5]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage

chat=ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.3)
messages=[
    SystemMessage(content="You are an expert in fashion design"),
    HumanMessage(content="Suggest what color t-shirt would go with beige pants")
]

response=chat(messages)

<ipython-input-5-1dc655d4dfda>:10: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=chat(messages)


In [6]:
print(response.content,end='\n')

A beige color is quite neutral and versatile, so you have many options for pairing it with a t-shirt. Here are a few color suggestions that would complement beige pants nicely:

1. Navy Blue: A classic and timeless choice that pairs well with beige for a sophisticated look.
2. Olive Green: A trendy and earthy tone that adds a touch of color without being too bold.
3. White: A crisp and clean option that creates a fresh and modern look when paired with beige.
4. Light Gray: A subtle and understated choice that creates a cohesive and polished outfit.
5. Dusty Pink: A soft and feminine color that adds a touch of warmth and complements beige beautifully.

Ultimately, the best color t-shirt to pair with beige pants will depend on your personal style and the occasion. Experiment with different colors to see what you feel most comfortable and confident in.


PROMPT TEMPLATES

In [7]:
from langchain import PromptTemplate

template="You are a class teacher, encourage your student based on their mood {mood}"

prompt=PromptTemplate(
    input_variables=["mood"],
    template=template
)

In [8]:
prompt.format(mood="sad")

'You are a class teacher, encourage your student based on their mood sad'

In [9]:
response2=chat(prompt.format(mood="sad"))

In [10]:
print(response2.content)

I can see that you are feeling sad today, and that's okay. It's important to remember that everyone has bad days sometimes. Just know that I am here for you and I care about you. Remember that it's okay to feel sad, but try to focus on the positive things in your life and know that this feeling won't last forever. You are strong and capable, and I believe in you. If you need someone to talk to or support, I am here for you. Keep your head up and know that brighter days are ahead.


Chains

In [11]:
from langchain import LLMChain

chain=LLMChain(llm=chat,prompt=prompt)

chain.run("happy")

<ipython-input-11-7cca744b1502>:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain=LLMChain(llm=chat,prompt=prompt)
<ipython-input-11-7cca744b1502>:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("happy")


'I am so proud of the positive energy and happiness you are bringing to class today! Your smile is contagious and it brightens up the whole room. Keep up the great work and continue to spread joy to those around you. Remember, a positive attitude can make all the difference in your day. Keep shining bright, you are doing amazing!'

In [12]:
from langchain.chains import SimpleSequentialChain
prompt2=PromptTemplate(
    input_variables=["marks"],
    template="Hello kids, these are your marks {marks}"
)

chain2=LLMChain(llm=chat,prompt=prompt2)

overall_chain=SimpleSequentialChain(chains=[chain,chain2],verbose=True)
print(overall_chain.run("happy"))



> Entering new SimpleSequentialChain chain...
I can see that you are in a great mood today! Your positive energy is contagious and it's wonderful to see you so happy. Keep up the good work and continue to spread that positivity to those around you. Remember, a positive attitude can make all the difference in achieving your goals and overcoming any challenges that come your way. Keep smiling and shining bright, you are capable of achieving great things!


Remember to always believe in yourself and never give up on your dreams. You are all capable of achieving amazing things and I have no doubt that you will succeed in whatever you set your mind to. Keep working hard, stay positive, and never lose sight of your goals. You are all capable of achieving greatness, so keep pushing forward and never stop believing in yourselves. I am so proud of each and every one of you. Keep up the great work and continue to shine bright!

> Finished chain.


Remember to always believe in yourself and nev

Chunk,embed,retrieve

In [13]:
explanation=chat("Explain quantum physics")

In [14]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=0
)
texts=text_splitter.create_documents([explanation.content])

In [15]:
texts

[Document(metadata={}, page_content='Quantum physics is a branch of physics that deals with the behavior of particles at the smallest'),
 Document(metadata={}, page_content='scales, such as atoms and subatomic particles. It is based on the principles of quantum mechanics,'),
 Document(metadata={}, page_content='which describes the behavior of particles in terms of probabilities and wave functions.'),
 Document(metadata={}, page_content='One of the key principles of quantum physics is the concept of superposition, which states that'),
 Document(metadata={}, page_content='particles can exist in multiple states at the same time until they are observed or measured. This'),
 Document(metadata={}, page_content='is in contrast to classical physics, where particles are assumed to have definite properties at all'),
 Document(metadata={}, page_content='times.'),
 Document(metadata={}, page_content='Another important concept in quantum physics is entanglement, which refers to the phenomenon where

In [16]:
texts[0].page_content

'Quantum physics is a branch of physics that deals with the behavior of particles at the smallest'

In [17]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [18]:
result=embeddings.embed_query(texts[1].page_content)

In [19]:
result

[0.0036541034933179617,
 -0.008750359527766705,
 -0.0052261799573898315,
 -0.02371107041835785,
 -0.004125076811760664,
 0.018644047901034355,
 -0.03606681153178215,
 -0.02676427736878395,
 -0.014720352366566658,
 -0.02163229137659073,
 -0.007996802218258381,
 0.010894100181758404,
 -0.005294390022754669,
 0.0001814871357055381,
 0.010127550922334194,
 -0.02052794024348259,
 0.017448749393224716,
 -0.0015688283601775765,
 -0.0048136720433831215,
 -0.013356153853237629,
 0.006275313440710306,
 0.021944109350442886,
 0.026816247031092644,
 -0.026868216693401337,
 -0.006658588536083698,
 0.03362424671649933,
 0.02728397212922573,
 -0.028531238436698914,
 0.0005814084433950484,
 -0.007821405306458473,
 0.013115795329213142,
 0.009094657376408577,
 -0.027335941791534424,
 -0.004267992917448282,
 -0.011901008896529675,
 0.02150236815214157,
 0.015084139071404934,
 -0.007827901281416416,
 -0.000799842644482851,
 -0.014083726331591606,
 0.020540934056043625,
 0.02267168089747429,
 0.0115956878

In [20]:
#!pip install pinecone

In [21]:
from langchain.vectorstores import Pinecone
import pinecone

index_name="langchain-start"
search=Pinecone.from_documents(texts,embeddings,index_name=index_name)


In [22]:
query="what is quantum physics?"
result=search.similarity_search(query)

In [23]:
result

[Document(metadata={}, page_content='Quantum physics is a branch of physics that deals with the behavior of particles at the smallest'),
 Document(metadata={}, page_content='Quantum physics has led to many groundbreaking discoveries and technologies, such as quantum'),
 Document(metadata={}, page_content='understanding of the nature of reality and the fundamental laws of physics.'),
 Document(metadata={}, page_content='scales, such as atoms and subatomic particles. It is based on the principles of quantum mechanics,')]

Agents

In [24]:
!pip install --upgrade langchain langchain-core langchain-openai


In [34]:
# Instead of importing from langchain_experimental.agents, try this:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_experimental.tools.python.tool import PythonREPL
from langchain.llms.openai import OpenAI

agent_executor = create_python_agent(
    llm=OpenAI(temperature=0, max_tokens=1000),
    tool=PythonREPLTool(),
    verbose=True
)

agent_executor.run("calculate the gravity of earth")

<ipython-input-34-81f5a497004d>:8: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm=OpenAI(temperature=0, max_tokens=1000),




> Entering new AgentExecutor chain...


 I can use the formula F = G * m1 * m2 / r^2 to calculate the gravity of earth.
Action: Python_REPL
Action Input: G = 6.67e-11
Observation: 
Thought: I need to define the mass of the earth and the distance from the center of the earth to the surface.
Action: Python_REPL
Action Input: m1 = 5.97e24
Observation: 
Thought: I need to convert the radius of the earth from km to m.
Action: Python_REPL
Action Input: r = 6371 * 1000
Observation: 
Thought: Now I can calculate the gravity of earth.
Action: Python_REPL
Action Input: F = G * m1 * m2 / r**2
Observation: NameError("name 'm2' is not defined")
Thought: I need to define the mass of the object.
Action: Python_REPL
Action Input: m2 = 1
Observation: 
Thought: Now I can calculate the gravity of earth.
Action: Python_REPL
Action Input: F = G * m1 * m2 / r**2
Observation: 
Thought: I now know the final answer
Final Answer: 9.819650000000001

> Finished chain.


'9.819650000000001'